In [1]:
# progen_viewer_reuse.py
import sys, os, random
sys.path.insert(0, r"D:\LLMDelivery-LJ\SimWorld")
sys.path.insert(0, r"D:\LLMDelivery-LJ\LLM-Delivery")  # 放 CityViewer 的工程根目录

from PyQt5.QtWidgets import QApplication
from simworld.map.map import Map, Node
from simworld.utils.vector import Vector

# 复用：通用底图可视化
from simworld.utils.city_viewer import CityViewer, build_city_map, thin_by_min_spacing_xy

# ---------------- 配置 ----------------
WORLD_JSON = r"D:\LLMDelivery-LJ\Test_Data\test\progen_world_enriched.json"
BUILDING_DEFS = r"D:\LLMDelivery-LJ\Test_Data\test\buildings.json"
CONFIG = {
    "map.input_roads": r"D:\LLMDelivery-LJ\Test_Data\test\roads.json",
    "traffic.sidewalk_offset": 100.0,
}

PATH_METHOD = "astar"   # "astar" | "greedy"
GREEDY_STEPS = 12
MIN_WP_SPACING_CM = 1500.0  # 路点最小间距（抽稀）

# --------- 简单贪心：朝终点更近的邻居前进 ---------
def greedy_next_points_towards(city_map: Map, start_node: Node, end_node: Node, steps=10):
    path = [start_node]
    cur = start_node
    visited = {start_node}
    adj = city_map.get_adjacency_list()
    for _ in range(steps):
        nbrs = adj.get(cur, [])
        if not nbrs:
            break
        cur_d = cur.position.distance(end_node.position)
        better = [nb for nb in nbrs if nb.position.distance(end_node.position) < cur_d]
        cand = better if better else nbrs
        cand_sorted = sorted(
            cand,
            key=lambda nb: (nb in visited, nb.position.distance(end_node.position))
        )
        cur = cand_sorted[0]
        if cur in visited:
            break
        visited.add(cur)
        path.append(cur)
    return path

# --------- 计算并绘制一条随机路径 ----------
def compute_and_draw(viewer: CityViewer, city_map: Map):
    # 随机两个“路口”节点
    start = city_map.get_random_node(type='intersection')
    end = city_map.get_random_node(type='intersection', exclude=[start])

    # A* / 贪心
    if PATH_METHOD.lower() == "astar":
        nodes = city_map.get_shortest_path(start, end)
        if nodes is None or len(nodes) < 2:
            nodes = greedy_next_points_towards(city_map, start, end, steps=GREEDY_STEPS)
    else:
        nodes = greedy_next_points_towards(city_map, start, end, steps=GREEDY_STEPS)

    route_xy = [(n.position.x, n.position.y) for n in (nodes or [])]
    route_xy = thin_by_min_spacing_xy(route_xy, MIN_WP_SPACING_CM)

    # 叠加到 Viewer（底图来自 CityViewer）
    viewer.clear_overlays()
    viewer.plot_route(route_xy, color="#00BCD4", width=3.0, scatter=True, show_endpoints=True)

    # 简单信息
    if route_xy:
        viewer.set_info_text(
            f"method={PATH_METHOD}, raw={len(nodes) if nodes else 0}, "
            f"thinned={len(route_xy)}, min_spacing={int(MIN_WP_SPACING_CM)}cm\n"
            f"start={route_xy[0]}  end={route_xy[-1]}"
        )
    else:
        viewer.set_info_text("No path.")

def main():
    app = QApplication(sys.argv)

    # 底图复用（建筑 + bus 路线在 CityViewer 内完成）
    viewer = CityViewer(
        world_json_path=WORLD_JSON,
        building_defs_path=BUILDING_DEFS,
        title="Progen Map (+ Dynamic Waypoints via CityViewer)",
        background_color="#F0F8FF",
        show_grid=True,
    )
    city_map = build_city_map(CONFIG)

    # 按钮：重算路径 / 保存图片
    viewer.add_button("Recompute Waypoints", lambda: compute_and_draw(viewer, city_map))
    viewer.add_button("Save Image", lambda: viewer.save_png(os.path.join(os.path.dirname(WORLD_JSON), "progen_map_output_with_waypoints.png")))

    # 先画一条
    compute_and_draw(viewer, city_map)

    viewer.show()
    sys.exit(app.exec_())

if __name__ == "__main__":
    main()


c:\Users\Shadow\miniconda3\envs\citynav\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SystemExit: 0

c:\Users\Shadow\miniconda3\envs\citynav\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
